#Import Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

from pandas import read_csv
from pandas import DataFrame
from pandas import concat

from pywt import dwt
from pywt import idwt
import pywt

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

from keras.layers import LSTM, Dense
from keras.models import Sequential
from tensorflow.keras import initializers

from openpyxl.workbook import Workbook
import pickle

tf.random.set_seed(42)

#Class Preprocessing

In [ ]:
class Preprocessing:

    def minmax_scale(df):
        FEATURES = ['Open', 'High', 'Low', 'Volume', 'Close']
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(np.array(df[FEATURES]))

        scaler_pred = MinMaxScaler()
        pred_scaled = scaler_pred.fit_transform(np.array(df['Close']).reshape(-1,1))
        return df_scaled, scaler_pred

    def inverse_scaler(pred, scaler):
        pred_inversed = scaler.inverse_transform(pred)
        return pred_inversed

    def splitting_data(df):
        test_size = 3
        train_size = len(df)-test_size
        train_data,test_data= df[0:train_size,:], df[train_size:len(df),:]
        return train_data, test_data


    def data_denoising(data,wavelet_type, threshold):
        decomposed_data = pd.DataFrame(index=range(len(data)))
        for i in range(data.shape[1]):
            # Mendekomposisi data menggunakan pywt.dwt
            cA, cD = pywt.dwt(data[:, i], wavelet_type)

            # Melakukan thresholding pada cD
            cD_threshold = pywt.threshold(cD, threshold, mode='soft')

            # Menggabungkan cA dan cD_threshold untuk merekonstruksi
            reconstructed_data = pywt.idwt(cA, cD_threshold, wavelet_type)

            # Menyimpan hasil rekonstruksi ke dalam dataframe hasil
            decomposed_data[i] = reconstructed_data

        return decomposed_data


    def create_dataset(dataset, time_step=1, index=4):
        dataX = []
        dataY = []

        for i in range(len(dataset)-time_step):
            dataX.append(dataset[i:(i+time_step)])
            dataY.append(float(dataset[i+time_step][index]))
        return np.array(dataX), np.array(dataY)


#Class Model

In [ ]:
class NeuralNetwork:
    def train_lstm(train_X, train_y, test_X, test_y, unit, epoch, batch):
        model = Sequential()
        model.add(LSTM(unit, kernel_initializer=initializers.GlorotUniform(seed=42),
                       input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(units=1, kernel_initializer=initializers.GlorotUniform(seed=42)))
        model.compile(loss='mae',optimizer='adam')
        history = model.fit(train_X, train_y, epochs=epoch, batch_size=batch,
                            validation_data=(test_X, test_y), verbose=2, shuffle=False)
        return model, history

    def save_model(model, category, stocks_name, hyperparam):
        if category == 0:
            model.save('LSTM_'+ stocks_name +str(hyperparam)+'.h5')
        else:
            model.save('DWT_LSTM'+ stocks_name +str(hyperparam)+'.h5')

#Class Evaluation

In [ ]:
class Evaluation:
    def rmse (y, yhat):
        differences = [y[i] - yhat[i] for i in range(len(y))]
        squared_differences = [d**2 for d in differences]
        sum_squared_differences = sum(squared_differences)
        mean_squared_error = sum_squared_differences / len(y)
        return (mean_squared_error**0.5)[0]

    def mae (y, yhat):
        differences = [y[i] - yhat[i] for i in range(len(y))]
        absolute_differences = [abs(x) for x in differences]
        sum_absolute_difference = sum(absolute_differences)
        mean_absolute_error = sum_absolute_difference / len(y)
        return mean_absolute_error[0]

    def mape (y, yhat):
        divided_differences = [abs((y[i] - yhat[i])/y[i]) for i in range(len(y))]
        sum_absolute_difference = sum(divided_differences)
        mean_absolute_percentage_error = sum_absolute_difference / len(y)
        return (mean_absolute_percentage_error*100)[0]

#Main

In [ ]:
BBCA_dfd= pd.read_csv('/content/drive/MyDrive/Data/BBCA.JK.csv')
BBCA_dfd

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-16,5260.0,5285.0,5245.0,5285.0,4813.968262,94972500
1,2019-01-17,5290.0,5330.0,5285.0,5330.0,4854.957520,91654000
2,2019-01-18,5340.0,5425.0,5315.0,5425.0,4941.490234,100873500
3,2019-01-21,5425.0,5600.0,5390.0,5545.0,5050.795410,87118000
4,2019-01-22,5600.0,5620.0,5450.0,5600.0,5100.893066,91907000
...,...,...,...,...,...,...,...
1226,2024-01-09,9600.0,9625.0,9575.0,9625.0,9625.000000,59848600
1227,2024-01-10,9650.0,9650.0,9550.0,9550.0,9550.000000,52774900
1228,2024-01-11,9625.0,9650.0,9575.0,9575.0,9575.000000,39381500
1229,2024-01-12,9650.0,9700.0,9600.0,9700.0,9700.000000,68253400


##Data Normalization

In [ ]:
x,scaler = Preprocessing.minmax_scale(BBCA_dfd)

In [ ]:
print('Data kelima fitur yang dinormalisasi : ')
print(x)

Data kelima fitur yang dinormalisasi : 
[[0.15428571 0.09897959 0.17116279 0.1313902  0.16223909]
 [0.16       0.10816327 0.17860465 0.1267992  0.17077799]
 [0.16952381 0.12755102 0.18418605 0.13955397 0.18880455]
 ...
 [0.98571429 0.98979592 0.97674419 0.05448254 0.97628083]
 [0.99047619 1.         0.98139535 0.09442552 1.        ]
 [1.         1.         1.         0.         1.        ]]


## Data Splitting

In [ ]:
train,test = Preprocessing.splitting_data(x)

In [ ]:
print("Training Data :")
print(train)
print("=========================================================")
print("Testing Data :")
print(test)

Training Data :
[[0.15428571 0.09897959 0.17116279 0.1313902  0.16223909]
 [0.16       0.10816327 0.17860465 0.1267992  0.17077799]
 [0.16952381 0.12755102 0.18418605 0.13955397 0.18880455]
 ...
 [0.98095238 0.98469388 0.95813953 0.07041233 0.97628083]
 [0.98095238 0.98469388 0.97674419 0.08279786 0.9857685 ]
 [0.99047619 0.98979592 0.97209302 0.07301171 0.971537  ]]
Testing Data :
[[0.98571429 0.98979592 0.97674419 0.05448254 0.97628083]
 [0.99047619 1.         0.98139535 0.09442552 1.        ]
 [1.         1.         1.         0.         1.        ]]


##Data Denoising

In [ ]:
threshold = 0.004
wavelet_type = 'db4'

denoised_train = Preprocessing.data_denoising(train,wavelet_type,threshold)

In [ ]:
print("Hasil data training yang telah bersih dari noise ")
print(denoised_train.values)
print(len(denoised_train.values))

Hasil data training yang telah bersih dari noise 
[[0.15444091 0.09757096 0.17128001 0.12973099 0.16079503]
 [0.15862625 0.10828911 0.17682462 0.12971233 0.17124551]
 [0.1702379  0.12866194 0.18587945 0.13665826 0.18818151]
 ...
 [0.97914184 0.98283911 0.96196319 0.07401206 0.97992598]
 [0.98369689 0.98749579 0.97389351 0.07997662 0.98296119]
 [0.98982205 0.98944009 0.97192297 0.07358438 0.97202076]]
1228


##Creating Dataset

In [ ]:
train_X, train_y = Preprocessing.create_dataset(denoised_train.values)
test_X, test_y = Preprocessing.create_dataset(test)

In [ ]:
print("Data train_X")
print("=====================================")
print(train_X)

Data train_X
[[[0.15444091 0.09757096 0.17128001 0.12973099 0.16079503]]

 [[0.15862625 0.10828911 0.17682462 0.12971233 0.17124551]]

 [[0.1702379  0.12866194 0.18587945 0.13665826 0.18818151]]

 ...

 [[0.95891976 0.9765894  0.95543382 0.11525239 0.97354568]]

 [[0.97914184 0.98283911 0.96196319 0.07401206 0.97992598]]

 [[0.98369689 0.98749579 0.97389351 0.07997662 0.98296119]]]


In [ ]:
print("Data train_y")
print("=====================================")
print(train_y)

Data train_y
[0.17124551 0.18818151 0.21426186 ... 0.97992598 0.98296119 0.97202076]


In [ ]:
print("Data test_X")
print("=====================================")
print(test_X)

Data test_X
[[[0.98571429 0.98979592 0.97674419 0.05448254 0.97628083]]

 [[0.99047619 1.         0.98139535 0.09442552 1.        ]]]


In [ ]:
print("Data test_y")
print("=====================================")
print(test_y)

Data test_y
[1. 1.]


## Creating Hyperparameter Combination

In [ ]:
hyperparams = []
batch = [16, 32]
epoch = [50, 100]
neuron = [20,50]
for j in batch:
    for k in epoch:
        for l in neuron:
            hyperparams.append((j,k,l))
hyperparams

[(16, 50, 20),
 (16, 50, 50),
 (16, 100, 20),
 (16, 100, 50),
 (32, 50, 20),
 (32, 50, 50),
 (32, 100, 20),
 (32, 100, 50)]

In [ ]:
hyperparam1 = hyperparams[:4]
hyperparam2 = hyperparams[4:8]
hyperparam1

[(16, 50, 20), (16, 50, 50), (16, 100, 20), (16, 100, 50)]

In [ ]:
hyperparam2

[(32, 50, 20), (32, 50, 50), (32, 100, 20), (32, 100, 50)]

##Training Model

In [ ]:
lstms1 = []
models1 = []
for batch, epoch, neuron in hyperparam1:
    model, lstm = NeuralNetwork.train_lstm(train_X, train_y, test_X, test_y, neuron, epoch, batch)
    lstms1.append(lstm)
    models1.append(model)

Epoch 1/50
77/77 - 4s - loss: 0.1455 - val_loss: 0.0142 - 4s/epoch - 51ms/step
Epoch 2/50
77/77 - 0s - loss: 0.0517 - val_loss: 0.0456 - 185ms/epoch - 2ms/step
Epoch 3/50
77/77 - 0s - loss: 0.0403 - val_loss: 0.0274 - 184ms/epoch - 2ms/step
Epoch 4/50
77/77 - 0s - loss: 0.0314 - val_loss: 0.0220 - 182ms/epoch - 2ms/step
Epoch 5/50
77/77 - 0s - loss: 0.0228 - val_loss: 0.0251 - 186ms/epoch - 2ms/step
Epoch 6/50
77/77 - 0s - loss: 0.0183 - val_loss: 0.0274 - 207ms/epoch - 3ms/step
Epoch 7/50
77/77 - 0s - loss: 0.0154 - val_loss: 0.0212 - 189ms/epoch - 2ms/step
Epoch 8/50
77/77 - 0s - loss: 0.0144 - val_loss: 0.0188 - 194ms/epoch - 3ms/step
Epoch 9/50
77/77 - 0s - loss: 0.0142 - val_loss: 0.0202 - 182ms/epoch - 2ms/step
Epoch 10/50
77/77 - 0s - loss: 0.0142 - val_loss: 0.0176 - 193ms/epoch - 3ms/step
Epoch 11/50
77/77 - 0s - loss: 0.0143 - val_loss: 0.0180 - 206ms/epoch - 3ms/step
Epoch 12/50
77/77 - 0s - loss: 0.0143 - val_loss: 0.0175 - 199ms/epoch - 3ms/step
Epoch 13/50
77/77 - 0s - lo

## Testing Model

In [ ]:
i=0
wb1 = Workbook()
ws1 = wb1.active
for m in models1:
    # make a prediction
    test_x2 = test_X
    yhat = m.predict(test_x2)
    inv_yhat = Preprocessing.inverse_scaler(yhat, scaler)
    inv_y = Preprocessing.inverse_scaler(test_y.reshape(-1,1), scaler)
    print(hyperparam1[i])
    print("Epoch: "+ str(lstms1[i].params['epochs']))
    print("Neurons: "+str(m.layers[0].units))

    i = i+1
    ws1['A'+str(i)] = 'DWT-LSTM'
    ws1['B'+str(i)] = hyperparam1[i-1][0]
    ws1['C'+str(i)] = hyperparam1[i-1][1]
    ws1['D'+str(i)] = hyperparam1[i-1][2]
    print('RMSE')
    print(Evaluation.rmse(inv_y,inv_yhat))
    ws1['E'+str(i)] = Evaluation.rmse(inv_y,inv_yhat)

    print('MAE')
    print(Evaluation.mae(inv_y,inv_yhat))
    ws1['F'+str(i)] = Evaluation.mae(inv_y,inv_yhat)

    print('MAPE')
    print(Evaluation.mape(inv_y,inv_yhat))
    ws1['G'+str(i)] = Evaluation.mape(inv_y,inv_yhat)

    NeuralNetwork.save_model(m, 1, 'BBCA',hyperparam1[i-1])
    with open('DWT_LSTM_BBCA'+str(hyperparam1[i-1])+'.pkl', 'wb') as f:
        pickle.dump(lstms1[i-1].history, f)
wb1.save('DWT_LSTM_BBCA_result1.xlsx')

1/1 [==============================] - 1s 891ms/step
(16, 50, 20)
Epoch: 50
Neurons: 20
RMSE
99.60382795379331
MAE
94.90185546875
MAPE
0.978369644007732


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 440ms/step
(16, 50, 50)
Epoch: 50
Neurons: 50
RMSE
69.63637561427886
MAE
59.20703125
MAPE
0.6103817654639175


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 429ms/step
(16, 100, 20)
Epoch: 100
Neurons: 20
RMSE
104.38477760476897
MAE
98.09423828125
MAPE
1.0112808070231958


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 428ms/step
(16, 100, 50)
Epoch: 100
Neurons: 50
RMSE
85.21420857911701
MAE
75.0751953125
MAPE
0.7739710856958762


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:

lstms2 = []
models2 = []
for batch, epoch, neuron in hyperparam2:
    model, lstm = NeuralNetwork.train_lstm(train_X, train_y, test_X, test_y, neuron, epoch, batch)
    lstms2.append(lstm)
    models2.append(model)

Epoch 1/50
39/39 - 3s - loss: 0.3207 - val_loss: 0.4832 - 3s/epoch - 66ms/step
Epoch 2/50
39/39 - 0s - loss: 0.0566 - val_loss: 0.0392 - 100ms/epoch - 3ms/step
Epoch 3/50
39/39 - 0s - loss: 0.1034 - val_loss: 0.0838 - 105ms/epoch - 3ms/step
Epoch 4/50
39/39 - 0s - loss: 0.0561 - val_loss: 0.0201 - 113ms/epoch - 3ms/step
Epoch 5/50
39/39 - 0s - loss: 0.0513 - val_loss: 0.0137 - 113ms/epoch - 3ms/step
Epoch 6/50
39/39 - 0s - loss: 0.0272 - val_loss: 0.0145 - 113ms/epoch - 3ms/step
Epoch 7/50
39/39 - 0s - loss: 0.0143 - val_loss: 0.0209 - 115ms/epoch - 3ms/step
Epoch 8/50
39/39 - 0s - loss: 0.0141 - val_loss: 0.0204 - 133ms/epoch - 3ms/step
Epoch 9/50
39/39 - 0s - loss: 0.0141 - val_loss: 0.0187 - 113ms/epoch - 3ms/step
Epoch 10/50
39/39 - 0s - loss: 0.0141 - val_loss: 0.0166 - 105ms/epoch - 3ms/step
Epoch 11/50
39/39 - 0s - loss: 0.0142 - val_loss: 0.0176 - 114ms/epoch - 3ms/step
Epoch 12/50
39/39 - 0s - loss: 0.0144 - val_loss: 0.0168 - 116ms/epoch - 3ms/step
Epoch 13/50
39/39 - 0s - lo

In [ ]:
i=0
wb2 = Workbook()
ws2 = wb2.active
for m in models2:
    # make a prediction
    test_x2 = test_X
    yhat = m.predict(test_x2)
    inv_yhat = Preprocessing.inverse_scaler(yhat, scaler)
    inv_y = Preprocessing.inverse_scaler(test_y.reshape(-1,1), scaler)
    print(hyperparam2[i])
    print("Epoch: "+ str(lstms2[i].params['epochs']))
    print("Neurons: "+str(m.layers[0].units))

    i = i+1
    ws2['A'+str(i)] = 'DWT-LSTM'
    ws2['B'+str(i)] = hyperparam2[i-1][0]
    ws2['C'+str(i)] = hyperparam2[i-1][1]
    ws2['D'+str(i)] = hyperparam2[i-1][2]

    print('RMSE')
    print(Evaluation.rmse(inv_y,inv_yhat))
    ws2['E'+str(i)] = Evaluation.rmse(inv_y,inv_yhat)

    print('MAE')
    print(Evaluation.mae(inv_y,inv_yhat))
    ws2['F'+str(i)] = Evaluation.mae(inv_y,inv_yhat)

    print('MAPE')
    print(Evaluation.mape(inv_y,inv_yhat))
    ws2['G'+str(i)] = Evaluation.mape(inv_y,inv_yhat)

    NeuralNetwork.save_model(m, 1, 'BBCA',hyperparam2[i-1])
    with open('DWT_LSTM_BBCA'+str(hyperparam2[i-1])+'.pkl', 'wb') as f:
        pickle.dump(lstms2[i-1].history, f)
wb2.save('DWT_LSTM_BBCA_result2.xlsx')

1/1 [==============================] - 0s 22ms/step
(32, 50, 20)
Epoch: 50
Neurons: 20
RMSE
111.09054458865496
MAE
107.60986328125
MAPE
1.1093800338273194
1/1 [==============================] - 0s 20ms/step
(32, 50, 50)
Epoch: 50
Neurons: 50
RMSE
49.585080928084444
MAE
38.43115234375
MAPE
0.39619744684278346
1/1 [==============================] - 0s 22ms/step
(32, 100, 20)
Epoch: 100
Neurons: 20
RMSE
104.37112636116638
MAE
99.7724609375
MAPE
1.0285820715206184


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 21ms/step
(32, 100, 50)
Epoch: 100
Neurons: 50
RMSE
89.65995819996633
MAE
82.83837890625
MAPE
0.8540039062500001


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
